In [2]:
%%bash
source ~/.profile # Make sure environment variables are loaded.
stop-connect-server.sh
start-connect-server.sh --packages org.apache.spark:spark-connect_2.12:$SPARK_VERSION

stopping org.apache.spark.sql.connect.service.SparkConnectServer
starting org.apache.spark.sql.connect.service.SparkConnectServer, logging to /opt/spark/logs/spark-jesus-org.apache.spark.sql.connect.service.SparkConnectServer-1-Karbranth.out


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import lower

SparkSession.builder.master("local[*]").getOrCreate().stop()

spark = SparkSession.builder.remote("sc://localhost:15002").getOrCreate()

AttributeError: 'NoneType' object has no attribute 'startswith'

In [6]:
# Cargar el archivo JSON y las palabras claves
df = spark.read.json("twitter/us_news_tweets.json")
pro_keywords = spark.read.text("pro_vax_keywords.txt")
anti_keywords = spark.read.text("anti_vax_keywords.txt")

In [7]:
# Tomar lo que queremos de los tweets
tweets = df.select("content","user.username","likeCount","retweetCount")

# Realizar el producto cruz para luego ver si las palabras estan en los tweets
producto_cruz_pro = tweets.crossJoin(pro_keywords)
tweets_pro_vax = producto_cruz_pro.filter(lower(col("content")).contains(col("value")))
producto_cruz_anti = tweets.crossJoin(anti_keywords)
tweets_anti_vax = producto_cruz_anti.filter(lower(col("content")).contains(col("value")))

tweets_pro_vax.show()
tweets_anti_vax.show()

+--------------------+---------------+---------+------------+----------+
|             content|       username|likeCount|retweetCount|     value|
+--------------------+---------------+---------+------------+----------+
|Kaixin Auto stock...|JacekWierzbicki|        0|           0|    health|
|Partnering with #...|  cryptovinesco|        0|           0| community|
|#Security #News \...|    E_4_Jupiter|        0|           0|    safety|
|#Security #News \...|    E_4_Jupiter|        0|           0|    health|
|Women on vegetari...|AsianTimeOnline|        0|           0|    health|
|RackWare Announce...|      AiThority|        0|           0|protection|
|Over 8000 VNC ins...|        AcooEdi|        0|           0|  research|
|Nitish Cabinet Ex...|      Scrilling|        0|           0|     trust|
|Horoscopes Aug. 1...|viralnews_osint|        0|           0|      care|
|Six (06) Covid-19...|      saranga09|        1|           1|    health|
|This Is What Scar...|     GafferNews|        0|   

In [ ]:
# Guardamos contadores de tweets pro y anti vacunas
numero_de_tweets_pro = tweets_pro_vax.count()
numero_de_tweets_anti = tweets_anti_vax.count()

In [12]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Definir el esquema del DataFrame
esquema = StructType([
    StructField("tipo", StringType(), True),
    StructField("numero", IntegerType(), True)
])
# Dataframe contadores
datos = [("pro-vacunas", numero_de_tweets_pro), ("anti-vacunas", numero_de_tweets_anti)]
conteo = spark.createDataFrame(datos, esquema)

conteo.show()

+----+------+
|tipo|numero|
+----+------+
| pro|426472|
|anti|121401|
+----+------+



In [ ]:
# Guardamos los resultados
tweets_pro_vax.write.json("salida_problema4_provax_spark")
tweets_anti_vax.write.json("salida_problema4_antivax_spark")

In [14]:
usuarios_anti = tweets_anti_vax.groupBy("username").count().orderBy(col("count").desc())
usuarios_pro = tweets_pro_vax.groupBy("username").count().orderBy(col("count").desc())

usuarios_anti.show()
usuarios_pro.show()

+---------------+-----+
|       username|count|
+---------------+-----+
|        poandpo| 5164|
|       ChronLaw| 4884|
|yourchoice_news| 4876|
|JacekWierzbicki| 3853|
|  MadrasTribune| 2166|
|    MarketsCafe| 2039|
|   EuropeNews_b| 1664|
|       smorffer| 1622|
|Knewz_Currently| 1541|
|RisksAssociated| 1465|
|    DonShelbyUK| 1370|
| robinsnewswire| 1283|
| NewsPlayerPlus| 1231|
|       riskmap_| 1143|
|  techinjektion| 1074|
|      UK_News_b| 1031|
|   etf_dividend| 1017|
|   chrisoldcorn| 1006|
|      condorbox|  867|
|   World_News_b|  750|
+---------------+-----+
only showing top 20 rows



<bound method DataFrame.show of DataFrame[username: string, count: bigint]>

In [ ]:
# Guardamos los resultados
conteo.write.json("conteo/total_tweets")
usuarios_anti.write.json("conteo/usuarios_anti")
usuarios_pro.write.json("conteo/usuarios_pro")